# Computation on NumPy Arrays: Funciones Universales

La computación en matrices NumPy puede ser muy rápida, o puede ser muy lenta.
La clave para hacerlo rápido es usar operaciones *vectorizadas*, generalmente implementadas a través de las *funciones universales* (ufuncs) de NumPy.
Esta sección motiva la necesidad de las ufuncs de NumPy, que pueden usarse para hacer cálculos repetidos sobre elementos de array mucho más eficientes.
A continuación, se presentan muchas de las ufuncs aritméticas más comunes y útiles disponibles en el paquete NumPy.

## La lentitud de los bucles

La implementación por defecto de Python (conocida como CPython) realiza algunas operaciones muy lentamente.
Esto se debe en parte a la naturaleza dinámica e interpretada del lenguaje: el hecho de que los tipos son flexibles, de modo que las secuencias de operaciones no pueden compilarse a código máquina eficiente como en lenguajes como C y Fortran.
Recientemente ha habido varios intentos de abordar esta debilidad: ejemplos bien conocidos son el proyecto [PyPy](http://pypy.org/), una implementación compilada just-in-time de Python; el proyecto [Cython](http://cython.org), que convierte código Python en código C compilable; y el proyecto [Numba](http://numba.pydata.org/), que convierte fragmentos de código Python en bytecode LLVM rápido.
Cada uno de ellos tiene sus puntos fuertes y débiles, pero se puede afirmar con seguridad que ninguno de los tres enfoques ha superado todavía el alcance y la popularidad del motor CPython estándar.

La relativa lentitud de Python se manifiesta generalmente en situaciones en las que se repiten muchas operaciones pequeñas - por ejemplo, bucles sobre matrices para operar sobre cada elemento.
Por ejemplo, imaginemos que tenemos una matriz de valores y queremos calcular el recíproco de cada uno.
Un enfoque sencillo podría ser el siguiente:

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values:np.array) -> np.array:
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

Esta implementación probablemente resulte bastante natural para alguien que tenga, por ejemplo, conocimientos de C o Java.
Pero si medimos el tiempo de ejecución de este código para una entrada grande, veremos que esta operación es muy lenta, ¡quizás sorprendentemente!
Vamos a comparar esto con la magia ``%timeit`` de IPython.

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.77 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Se tardan varios segundos en calcular ese millón de operaciones y almacenar el resultado.
Cuando incluso los teléfonos móviles tienen velocidades de procesamiento medidas en Giga-FLOPS (es decir, miles de millones de operaciones numéricas por segundo), esto parece casi absurdamente lento.
Resulta que el cuello de botella aquí no son las operaciones en sí, sino la comprobación de tipos y los envíos de funciones que CPython debe hacer en cada ciclo del bucle.
Cada vez que se calcula el recíproco, Python primero examina el tipo del objeto y hace una búsqueda dinámica de la función correcta a utilizar para ese tipo.
Si estuviéramos trabajando en código compilado, esta especificación de tipo sería conocida antes de que el código se ejecute y el resultado podría ser calculado mucho más eficientemente.

## Presentación de UFuncs

Para muchos tipos de operaciones, NumPy proporciona una interfaz conveniente para este tipo de rutinas compiladas y tipadas estáticamente. Esto se conoce como una operación *vectorizada*.
Esto se puede lograr simplemente realizando una operación en la matriz, que luego se aplicará a cada elemento.
Este enfoque vectorizado está diseñado para empujar el bucle a la capa compilada que subyace en NumPy, lo que lleva a una ejecución mucho más rápida.

Compare los resultados de los dos siguientes:

In [3]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


Si observamos el tiempo de ejecución de nuestro gran array, vemos que se completa órdenes de magnitud más rápido que el bucle de Python:

In [4]:
%timeit (1.0 / big_array)

2.14 ms ± 149 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Las operaciones vectoriales en NumPy se implementan mediante *ufuncs*, cuyo principal propósito es ejecutar rápidamente operaciones repetidas sobre valores en arrays de NumPy.
Los ufuncs son extremadamente flexibles - antes vimos una operación entre un escalar y un array, pero también podemos operar entre dos arrays:

In [5]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

Además, las operaciones ufunc no se limitan a las matrices unidimensionales, sino que también pueden actuar sobre matrices multidimensionales:

In [6]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

Los cálculos que utilizan la vectorización a través de ufuncs son casi siempre más eficientes que su contrapartida implementada utilizando bucles de Python, especialmente a medida que las matrices crecen en tamaño.
Cada vez que veas un bucle de este tipo en un script de Python, deberías considerar si se puede sustituir por una expresión vectorizada.

## Explorando las UFuncs de NumPy

Ufuncs existen en dos formas: *unary ufuncs*, que operan en una sola entrada, y *binary ufuncs*, que operan en dos entradas.
Aquí veremos ejemplos de ambos tipos de funciones.

### Aritmética de matrices

Los ufuncs de NumPy se sienten muy naturales de usar porque hacen uso de los operadores aritméticos nativos de Python.
Se pueden utilizar la suma, resta, multiplicación y división estándar:

In [7]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # división entera

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


También hay un ufunc para la negación, y un operador ``**`` para la exponenciación, y un operador ``%`` para el módulo:

In [8]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


Además, se pueden encadenar como se desee y se respeta el orden estándar de las operaciones:

In [9]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

Cada una de estas operaciones aritméticas son simplemente envolturas convenientes alrededor de funciones específicas incorporadas en NumPy; por ejemplo, el operador ``+`` es una envoltura para la función ``add``:

In [10]:
np.add(x, 2)

array([2, 3, 4, 5])

La siguiente tabla enumera los operadores aritméticos implementados en NumPy:

| Operador	    | Ufunc equivalente    | Descripción                          |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Adición (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Resta (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Negación unaria (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplicación (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |División (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |División entera (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponenciación (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Módulo/remanente (e.g., ``9 % 4 = 1``)|

Además, existen operadores booleanos/bitwise.

### Valor absoluto

Al igual que NumPy entiende los operadores aritméticos incorporados en Python, también entiende la función de valor absoluto incorporada en Python:

In [11]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

La correspondiente ufunc de NumPy es ``np.absolute``, que también está disponible con el alias ``np.abs``:

In [12]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [13]:
np.abs(x)

array([2, 1, 0, 1, 2])

Esta ufunc también puede manejar datos complejos, en los que el valor absoluto devuelve la magnitud:

In [14]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Funciones trigonométricas

NumPy proporciona un gran número de ufunciones útiles, y algunas de las más útiles para el científico de datos son las funciones trigonométricas.
Empezaremos definiendo un array de ángulos:

In [15]:
theta = np.linspace(0, np.pi, 3)

Ahora podemos calcular algunas funciones trigonométricas sobre estos valores:

In [16]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


Los valores se calculan con precisión de máquina, por lo que los valores que deberían ser cero no siempre llegan exactamente a cero.
También se dispone de funciones trigonométricas inversas:

In [17]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponentes y logaritmos

Otro tipo común de operación disponible en un ufunc de NumPy son los exponenciales:

In [18]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


La inversa de los exponenciales, los logaritmos, también están disponibles.
El ``np.log`` básico da el logaritmo natural; si prefiere calcular el logaritmo de base-2 o el logaritmo de base-10, también están disponibles:

In [19]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


También hay algunas versiones especializadas que son útiles para mantener la precisión con entradas muy pequeñas:

In [20]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


Cuando ``x`` es muy pequeño, estas funciones dan valores más precisos que si se utilizara ``np.log`` o ``np.exp`` en bruto.

### Specialized ufuncs

NumPy tiene muchas más ufuncs disponibles, incluyendo funciones trigonométricas hiperbólicas, aritmética bitwise, operadores de comparación, conversiones de radianes a grados, redondeo y restos, y mucho más.
La documentación de NumPy contiene muchas funciones interesantes.

Otra fuente excelente para ufuncs más especializados y oscuros es el submódulo ``scipy.special``.
Si quieres calcular alguna función matemática oscura en tus datos, es probable que esté implementada en ``scipy.special``.
Hay demasiadas funciones para enumerarlas todas, pero el siguiente fragmento muestra un par que podrían surgir en un contexto estadístico:

In [21]:
from scipy import special

In [22]:
# Funciones gamma (factoriales generalizados) y funciones afines
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [23]:
# Función de error (integral de Gauss)
# su complemento, y su inversa
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


Hay muchas, muchas más ufuncs disponibles tanto en NumPy como en ``scipy.special``.
Dado que la documentación de estos paquetes está disponible en línea, una búsqueda en la web del tipo "gamma function python" generalmente encontrará la información relevante.

## Características avanzadas de Ufunc

Muchos usuarios de NumPy hacen uso de ufuncs sin llegar a aprender su conjunto completo de características.
Vamos a esbozar algunas características especializadas de ufuncs aquí.

### Especificar salida

Para cálculos grandes, a veces es útil poder especificar el array donde se almacenará el resultado del cálculo.
En lugar de crear un array temporal, esto se puede utilizar para escribir los resultados del cálculo directamente en la ubicación de memoria donde se desea que estén.
Para todas las ufuncs, esto puede hacerse utilizando el argumento ``out`` de la función:

In [24]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]


Esto puede utilizarse incluso con vistas de matrices. Por ejemplo, podemos escribir los resultados de un cálculo en uno de cada dos elementos de una matriz especificada:

In [25]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


Si en lugar de eso hubiéramos escrito ``y[::2] = 2 ** x``, se habría creado una matriz temporal para contener los resultados de ``2 ** x``, seguida de una segunda operación para copiar esos valores en la matriz ``y``.
Esto no supone una gran diferencia para un cálculo tan pequeño, pero para matrices muy grandes el ahorro de memoria derivado del uso cuidadoso del argumento ``out`` puede ser significativo.

### Agregados

Para ufuncs binarios, hay algunos agregados interesantes que se pueden calcular directamente desde el objeto.
Por ejemplo, si queremos *reducir* un array con una operación concreta, podemos utilizar el método ``reduce`` de cualquier ufunc.
Una reducción aplica repetidamente una operación dada a los elementos de un array hasta que sólo queda un único resultado.

Por ejemplo, llamar a ``reduce`` en la ufunc ``add`` devuelve la suma de todos los elementos del array:

In [26]:
x = np.arange(1, 6)
np.add.reduce(x)

np.int64(15)

Del mismo modo, al llamar a ``reduce`` en la ufunc ``multiply`` se obtiene el producto de todos los elementos del array:

In [27]:
np.multiply.reduce(x)

np.int64(120)

Si queremos almacenar todos los resultados intermedios del cálculo, podemos utilizar ``accumulate``:

In [28]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [29]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

Tenga en cuenta que para estos casos particulares, hay funciones NumPy dedicadas para calcular los resultados (``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``).

### Productos exteriores

Por último, cualquier ufunc puede calcular la salida de todos los pares de dos entradas diferentes utilizando el método ``outer``.
Esto te permite, en una línea, hacer cosas como crear una tabla de multiplicar:

In [30]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

Otra característica extremadamente útil de ufuncs es la capacidad de operar entre matrices de diferentes tamaños y formas, un conjunto de operaciones conocido como *broadcasting*.

<!--NAVIGATION-->
< [Fundamentos de Numpy Arrays](2-Fundamentos_de_Numpy_Arrays.ipynb) | [Cálculo sobre agregados de matrices](4-Calculo_sobre_agregados_de_matrices.ipynb) >
